In [1]:
%load_ext autoreload
%autoreload 2

import re
import os
import sys
import requests


from bs4 import BeautifulSoup

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from name import wiki
from name.named_entity import NamedEntity

# The Wiki List

Starting from the Wiki list page: https://en.wikipedia.org/wiki/List_of_towns_in_Japan



In [11]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_towns_in_Japan')
page = BeautifulSoup(res.text)

In [17]:
rows = page.select('.mw-parser-output .wikitable tbody tr:has(td)')
print('rows', len(rows))

rows[-3:]

rows 744


[<tr>
 <td align="left"><a href="/wiki/Yaese,_Okinawa" title="Yaese, Okinawa">Yaese</a></td>
 <td align="left"><span style="font-size:125%;"><span lang="ja" title="Japanese language text">八重瀬町</span></span></td>
 <td align="left">Okinawa</td>
 <td align="left"><a href="/wiki/Shimajiri_District,_Okinawa" title="Shimajiri District, Okinawa">Shimajiri</a></td>
 <td>26.96
 </td></tr>, <tr>
 <td align="left"><a href="/wiki/Taketomi,_Okinawa" title="Taketomi, Okinawa">Taketomi</a></td>
 <td align="left"><span style="font-size:125%;"><span lang="ja" title="Japanese language text">竹富町</span></span></td>
 <td align="left">Okinawa</td>
 <td align="left"><a href="/wiki/Yaeyama_District,_Okinawa" title="Yaeyama District, Okinawa">Yaeyama</a></td>
 <td>334.39
 </td></tr>, <tr>
 <td align="left"><a href="/wiki/Yonaguni,_Okinawa" title="Yonaguni, Okinawa">Yonaguni</a></td>
 <td align="left"><span style="font-size:125%;"><span lang="ja" title="Japanese language text">与那国町</span></span></td>
 <td align

In [27]:
row = rows[0]
print('English: ', row.select('td')[0].text)
print('Japanese: ', row.select('td')[1].text.replace('町', ''))
print('Src: ', 'https://en.wikipedia.org' + row.select('td a')[0].get('href'))

English:  Nanporo
Japanese:  南幌
Src:  https://en.wikipedia.org/wiki/Nanporo,_Hokkaido


In [28]:
entities = []

for row in rows:
    src_url = 'https://en.wikipedia.org' + row.select('td a')[0].get('href')
    en_name = row.select('td')[0].text
    jp_names = [
        row.select('td')[1].text.replace('町', ''),
        row.select('td')[1].text
    ]
    
    entities += [NamedEntity(src_url, english_names=en_name, japanese_names=jp_names)]

print('entities', len(entities))
entities[-3:]

entities 744


[<Entity src_url=https://en.wikipedia.org/wiki/Yaese,_Okinawa (Yaese, 八重瀬)>,
 <Entity src_url=https://en.wikipedia.org/wiki/Taketomi,_Okinawa (Taketomi, 竹富)>,
 <Entity src_url=https://en.wikipedia.org/wiki/Yonaguni,_Okinawa (Yonaguni, 与那国)>]

In [29]:
downloaded_filename = '../data/towns.csv'
with open(downloaded_filename, 'w') as fp:
    NamedEntity.to_csv(entities, fp)